# Aggregated Poverty Rates



In [1]:
%load_ext autoreload
%autoreload 1
%aimport censuslib.dataframe

In [2]:
from censuslib.dataframe import CensusDataFrame
import pandas as pd
from ambry import get_library
l = get_library()
b = l.bundle('census.gov-acs_p5ye2014-poverty-hdp-0.0.1')

In [3]:

p = l.partition('census.gov-acs-p5ye2014-b17020')

w = b.warehouse('child_poverty')

w.query("""

-- Get only counties in California

CREATE TABLE counties AS 
SELECT gvid, name AS county_name FROM census.gov-tiger-2015-counties
WHERE statefp = 6;

INDEX counties (gvid);


""").close()

Failed to index p04M02C002; SQLError: no such table: main.counties


In [4]:
# Select only California counties
def pred(r):
    from geoid.civick import GVid
    return (r.stusab == 'ca' and GVid.parse(r.gvid, exception=False).level == 'county')

from censuslib.dataframe import CensusDataFrame

df = CensusDataFrame(p.dataframe(pred))


In [5]:
male_total_cols = df.dim_columns("age == 'na' ")
male_cols = df.dim_columns("age != 'na' ")
print male_total_cols, male_cols

[u'b17020001', u'b17020002'] [u'b17020003', u'b17020004', u'b17020005', u'b17020006', u'b17020007', u'b17020008', u'b17020009', u'b17020010', u'b17020011', u'b17020012', u'b17020013', u'b17020014', u'b17020015', u'b17020016', u'b17020017']


In [6]:
%aimport censuslib.dataframe
from censuslib.dataframe import CensusDataFrame
df = CensusDataFrame(df)
df2 = df[['gvid','b17020001','b17020002','b17020002_m90', 'b17020010','b17020010_m90']].copy()
df2['all_below'], df2['all_below_m90'] = df.sum_col_group(3, 9)
df2['all_above'], df2['all_above_m90'] = df.sum_col_group(11, 17)
df2.head()

,gvid,b17020001,b17020002,b17020002_m90,b17020010,b17020010_m90,all_below,all_below_m90,all_above,all_above_m90
0,0O0601,1531346,197191,4256,1334155,4578,197191,3312.377545,1334155,4381.812867
1,0O0603,1184,179,72,1005,160,179,68.095521,1005,149.672977
2,0O0605,32932,4277,732,28655,827,4277,544.889897,28655,822.734465
3,0O0607,216592,46643,1844,169949,1883,46643,1632.086701,169949,2069.945410
4,0O0609,44223,5352,776,38871,802,5352,645.483540,38871,952.678330


In [8]:
df3 = df[['gvid','b17020001','b17020002','b17020002_m90', 'b17020010','b17020010_m90']].copy()
df3['all_below'], df3['all_below_m90'] = df.sum_m(df.dim_columns(" age != 'na' and col_num < 10 "))
df3['all_above'], df3['all_above_m90'] = df.sum_m(df.dim_columns(" age != 'na' and col_num > 10 "))
df3.head()

,gvid,b17020001,b17020002,b17020002_m90,b17020010,b17020010_m90,all_below,all_below_m90,all_above,all_above_m90
0,0O0601,1531346,197191,4256,1334155,4578,197191,3312.377545,1334155,4381.812867
1,0O0603,1184,179,72,1005,160,179,68.095521,1005,149.672977
2,0O0605,32932,4277,732,28655,827,4277,544.889897,28655,822.734465
3,0O0607,216592,46643,1844,169949,1883,46643,1632.086701,169949,2069.945410
4,0O0609,44223,5352,776,38871,802,5352,645.483540,38871,952.678330


In [11]:
df2.equals(df3)

True